In [ ]:

import pandas as pd

# Load the datasets
path_fake = "/content/drive/MyDrive/Colab Notebooks/News _dataset/Fake.csv"
data_fake = pd.read_csv(path_fake)

path_real = "/content/drive/MyDrive/Colab Notebooks/News _dataset/True.csv"
data_real = pd.read_csv(path_real)

# Add a label column to each dataset
data_fake['label'] = 1
data_real['label'] = 0

# Combine the datasets into a single dataframe
data_combined = pd.concat([data_fake, data_real], ignore_index=True)

# Shuffle the combined dataset to mix fake and real news articles
data_combined = data_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the first few rows of the combined dataset
data_combined.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",1
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",0
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",1
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",0


In [ ]:
data_new=data_combined.sample(n=11000,random_state=42).reset_index(drop=True)
data_test = data_new.sample(n=1000, random_state=42).reset_index(drop=True)
data_new = data_new.drop(data_test.index).reset_index(drop=True)
# data_test=data_combined.sample(n=16).reset_index(drop=True)
# data_new=data_combined.iloc[1:51,:]
# data_test=data_combined.iloc[51:56,:]

In [ ]:
model_name = "bert-base-cased"
!pip install transformers
import pandas as pd
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
##Set random values
seed_val = 40
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)
transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 300   #length of a sequential sentence
batch_size = 32      #Determines the number of samples hat will be passed through to the network at one time.


num_hidden_layers_d = 1;

# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.2     #randomly selected neurons are ignored during training

# Replicate labeled data to balance poorly represented datasets,
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-5

epsilon = 1e-8      #shows the change in o/p when a single sample is executed/
num_train_epochs = 10
multi_gpu = True             #It is supposed to run in single gpu
# Scheduler to run the tasks at a specific time.
apply_scheduler = False
warmup_proportion = 0.1       #Its used to indicate set of training steps with very low learning rate.
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------

model_name = "bert-base-cased"
#model_name = "bert-base-uncased"
#model_name = "roberta-base"
#model_name = "albert-base-v2"
#model_name = "xlm-roberta-base"
#model_name = "amazon/bort"



fatal: destination path 'ganbert' already exists and is not an empty directory.


In [ ]:

import torch
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)

        layers = []
        hidden_sizes = [input_size] + hidden_sizes

        for i in range(len(hidden_sizes) - 1):
            layers.extend([
                nn.Conv1d(in_channels=hidden_sizes[i], out_channels=hidden_sizes[i + 1], kernel_size=1),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Dropout(dropout_rate)
            ])

        self.layers = nn.Sequential(*layers)
        self.logit = nn.Linear(hidden_sizes[-1], num_labels)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        # Input shape: [batch_size, sequence_length, hidden_size]
        input_rep = input_rep.transpose(1, 2)  # Change to [batch_size, hidden_size, sequence_length]
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        last_rep = torch.mean(last_rep, dim=-1)  # Aggregate along sequence length
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs

In [ ]:
# The config file is required to get the dimension of the vector produced by
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)
# Define the number and width of hidden layers

hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

discriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=2, dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():
  # generator.cuda()
  discriminator.cuda()
  transformer.cuda()
  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)

In [ ]:
input_ids=[]
for text in data_new['title']:
  encoded_sent = tokenizer.encode(text, add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
  input_ids.append(encoded_sent)
input_ids = torch.tensor(input_ids)
dataloader1 = DataLoader(input_ids, batch_size=32, shuffle=True)
input_ids_test=[]
for text in data_test['title']:
  encoded_sent = tokenizer.encode(text, add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
  input_ids_test.append(encoded_sent)
input_ids_test = torch.tensor(input_ids_test)
test_dataloader= DataLoader(input_ids_test, batch_size=32, shuffle=True)

In [ ]:
import torch.nn.functional as F
import torch.optim as optim

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Learning rate
lr = 1e-4

# Move models to the GPU
transformer.to(device)
discriminator.to(device)

# Optimizer for the discriminator
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr)

# Number of epochs
num_epochs = 8

for epoch_i in range(num_epochs):
    loss1 = 0
    i = 0  # Reset the index for each epoch

    # Set the discriminator to training mode
    discriminator.train()


    for batch in dataloader1:
        # If batch is a list of tensors, stack them into a single tensor
        if isinstance(batch, list):
            batch = torch.stack(batch).to(device)
        else:
            batch = batch.to(device)

        batch_size = batch.size(0)

        # Model outputs
        model_outputs = transformer(batch)
        hidden_states = model_outputs.last_hidden_state
        # pooled_hidden_states = torch.mean(hidden_states, dim=1)

        features, logits, probs = discriminator(hidden_states)

        # Calculate log probabilities
        log_probs = F.log_softmax(logits, dim=-1)

        # Convert the labels to a PyTorch tensor before one-hot encoding and move to GPU
        labels = torch.tensor(data_new.iloc[i:i+batch_size, -1].values, dtype=torch.long).to(device)
        label2one_hot = F.one_hot(labels, num_classes=2).float().to(device)

        # Calculate per example loss
        per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)

        ans = torch.mean(per_example_loss)
        loss1 += ans.item()

        # Discriminator loss
        d_loss = ans

        # Zero the parameter gradients
        optimizer_d.zero_grad()

        # Backward pass
        d_loss.backward()

        # Optimization step
        optimizer_d.step()

        # Update the index
        i += batch_size

    # Calculate the average loss
    avg_loss = loss1 / len(dataloader1)
    print(f"Epoch {epoch_i + 1}: avg_loss: {avg_loss}")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1: avg_loss: 0.6989065081166764
Epoch 2: avg_loss: 0.6975435130893232
Epoch 3: avg_loss: 0.6959668413137856
Epoch 4: avg_loss: 0.6953147664999428


KeyboardInterrupt: 

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set the discriminator to evaluation mode
discriminator.eval()

# List to store the true labels and predicted labels
true_labels = []
predicted_labels = []
i=0
with torch.no_grad():
    for batch in test_dataloader:
        # If batch is a list of tensors, stack them into a single tensor
        if isinstance(batch, list):
            batch = torch.stack(batch).to(device)
        else:
            batch = batch.to(device)

        batch_size = batch.size(0)

        # Model outputs
        model_outputs = transformer(batch)
        hidden_states = model_outputs.last_hidden_state
        features, logits, probs = discriminator(hidden_states)

        # Get the predicted labels
        _, preds = torch.max(logits, dim=1)

        # Convert the labels to a PyTorch tensor and move to GPU
        labels = torch.tensor(data_test.iloc[i:i+batch_size, -1].values, dtype=torch.long).to(device)

        # Append the true labels and predictions to the lists
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())
        i+=batch_size

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

Test Accuracy: 0.5290
Test Precision: 0.2798
Test Recall: 0.5290
Test F1 Score: 0.3660


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pickle
# File path
file_path = 'data.pkl'

# Serialize and save to pickle file
with open(file_path, 'wb') as f:
    pickle.dump(discriminator, f)
